In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import sys
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
df=pd.read_csv('Book1-Opp Data2.csv',parse_dates=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 35 columns):
ID                                                  100 non-null int64
NAME                                                100 non-null object
CREATEDDATE                                         100 non-null object
EXPECTEDCLOSUREDATE                                 100 non-null object
ACTIVITYDATE                                        100 non-null object
CLOSURE_DATE                                        34 non-null object
OPPORTUNITYSTATUS                                   100 non-null object
HASCOMPETITORS                                      100 non-null object
EXPECTEDTOTAL                                       100 non-null int64
PROBABILITY                                         100 non-null int64
COMPANY                                             100 non-null object
EMAIL                                               100 non-null object
LEADSOURCE                          

In [4]:
df['CREATEDDATE']=pd.to_datetime(df['CREATEDDATE'])
df['EXPECTEDCLOSUREDATE']=pd.to_datetime(df['EXPECTEDCLOSUREDATE'])
df['ACTIVITYDATE']=pd.to_datetime(df['ACTIVITYDATE'])

In [5]:
np.random.seed(0)

In [6]:
def fnc1(row):
    if row['OPPORTUNITYSTATUS']=='5 - Closed Lost':
        val = 'No'
    elif row['OPPORTUNITYSTATUS']=='5 - Closed Won':
        val = 'Yes'
    else:
        val = 'Open'
    return val
df['Converted']=df.apply(fnc1, axis=1)
df['is_train']=np.where(df['Converted']!='Open',True,False ) 
print ('training in df:',len(df[df['is_train']==True]))
df2 = df.loc[df['is_train'] == False]
df2=df2.drop('is_train',1)
df2['is_train']=np.random.uniform(0, 1, len(df2)) <= .5
df_new = pd.concat([df.loc[df['is_train'] == True], df2])

training in df: 34


In [7]:
train,test = df_new[df_new['is_train']==True],df_new[df_new['is_train']==False]
print ('training data:',len(train))
print ('test data:',len(test))
test2 = test

training data: 66
test data: 34


In [8]:
features = df_new[['EXPECTEDTOTAL','STATE','ELAPSED_TIME']].columns
features

Index(['EXPECTEDTOTAL', 'STATE', 'ELAPSED_TIME'], dtype='object')

In [9]:
y = pd.factorize(train['Converted'])[0]
print(train.loc[:,['ID','Converted','EXPECTEDTOTAL','STATE','ELAPSED_TIME']])

     ID Converted  EXPECTEDTOTAL STATE  ELAPSED_TIME
0     1       Yes           2000    FL            16
1     2       Yes           1500    NC            11
2     3       Yes           1300    MO            13
3     4       Yes           3500    CA            16
4     5       Yes           1800    AR             6
10   11       Yes           1900    FL            13
11   12       Yes            500    NC             9
12   13       Yes            800    MO             3
13   14       Yes           4300    CA            14
14   15       Yes           1100    AR             7
19   20       Yes           2000    AR             6
20   21        No           2500    FL            31
21   22        No           3000    NC            31
22   23        No           3500    MO            35
24   25        No           5300    AR            47
25   26       Yes           3800    CA            30
27   28       Yes           2000    MO            14
28   29       Yes            500    CA        

In [10]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
      dtype=int64)

In [11]:
le = preprocessing.LabelEncoder()
clf=RandomForestClassifier(n_jobs=2,random_state=0)
train = train[features]
train.info()
for column_name in train[features].columns:
    if train[column_name].dtype == object:
        train[column_name] = le.fit_transform(train[column_name].astype(str))
    else:
        pass
clf.fit(train[features],y)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 0 to 99
Data columns (total 3 columns):
EXPECTEDTOTAL    66 non-null int64
STATE            66 non-null object
ELAPSED_TIME     66 non-null int64
dtypes: int64(2), object(1)
memory usage: 2.1+ KB


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [12]:
test = test.loc[:,features]
test.info()
for column_name in test[features].columns:
    if test[column_name].dtype == object:
        test[column_name] = le.fit_transform(test[column_name].astype(str))
    else:
        pass
clf.predict(test[features])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 5 to 96
Data columns (total 3 columns):
EXPECTEDTOTAL    34 non-null int64
STATE            34 non-null object
ELAPSED_TIME     34 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.1+ KB


array([2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [13]:
print (test2.loc[:,['ID','OPPORTUNITYSTATUS','EXPECTEDTOTAL','STATE','ELAPSED_TIME']])

    ID  OPPORTUNITYSTATUS  EXPECTEDTOTAL STATE  ELAPSED_TIME
5    6       1 - Prospect           4300    CA            30
6    7       1 - Prospect           5800    NC            26
7    8       1 - Prospect           6300    MO            21
8    9  2 - Qualification           7000    CA            16
15  16       1 - Prospect           1300    CA            30
17  18       1 - Prospect          11000    MO            21
18  19       1 - Prospect          13000    CA            16
26  27       1 - Prospect           6300    NC            26
29  30       1 - Prospect            300    AR             6
33  34       3 - Proposal           5800    CA            16
34  35       1 - Prospect           6300    AR             6
42  43       3 - Proposal           4800    MO            21
44  45       3 - Proposal           5800    AR             6
45  46       1 - Prospect           6300    CA            30
47  48       3 - Proposal           2500    MO            21
49  50       3 - Proposa

In [19]:
y=df_new['Converted']
X=df_new.loc[:,['EXPECTEDTOTAL','STATE','ELAPSED_TIME']]
indices = df_new.index.values
X_train, X_test, indices_train,indices_test = train_test_split(X, indices, test_size=0.3) 
y_train, y_test = y[indices_train],  y[indices_test]
clf2=RandomForestClassifier(n_estimators=100)
for column_name in X_train.columns:
    if X_train[column_name].dtype == object:
        X_train[column_name] = le.fit_transform(X_train[column_name].astype(str))
    else:
        pass
clf2.fit(X_train,y_train)

C:\Users\vivkashy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [20]:
for column_name in X_test.columns:
    if X_test[column_name].dtype == object:
        X_test[column_name] = le.fit_transform(X_test[column_name].astype(str))
    else:
        pass
y_pred=clf2.predict(X_test)

C:\Users\vivkashy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8333333333333334


In [22]:
df_results = X_test.copy()
df_results.loc[indices_test,'prediction']=y_pred
df3 = pd.merge(df_results, df_new, left_index=True, right_index=True)
df3.loc[:,['EXPECTEDTOTAL_y','STATE_y','ELAPSED_TIME_y','Converted','prediction']]

,EXPECTEDTOTAL_y,STATE_y,ELAPSED_TIME_y,Converted,prediction
81,5300,NC,26,Open,Open
21,3000,NC,31,No,Open
84,2000,AR,6,Yes,Yes
93,11000,CA,16,Open,Open
28,500,CA,13,Yes,Yes
80,4800,FL,30,Open,Open
73,9000,CA,16,Open,Open
64,6300,AR,6,Open,Open
71,5800,NC,26,Open,Open
30,3500,FL,31,No,Open
